# Ego net downloader

COSC2671 Social Media and Network Analytics

Lukas Krodinger, 3931415

In [24]:
from twitterClient import twitterClient
import math
import networkx as nx
import json

In [25]:
client = twitterClient() # Only used for getting user id

In [26]:
# Assigns a logarithmic value depending on the follower count
def calculate_importance(follower_count):
    if follower_count <= 0:
        follower_count = 1
    return math.ceil(math.log10(follower_count))

def load(filename):
    datas = []
    with open(filename, 'r') as f:
        for line in f:
            data = json.loads(line)
            datas.append(data)
    return datas

Parameters:

In [27]:
# Username of the ego
username_or_id = "CricketAus"

# Where to store the data
followers_filename = 'filtered_followers.json'
following_filename = 'filtered_following.json'
graph_filename= 'ego.graphml'

# ego attributes
user =  client.get_user(username_or_id)
ego_id = user.data.id
ego_username = user.data.username_or_id
ego_name = user.data.name

In [28]:
# Create graph
ego_graph = nx.DiGraph()
ego_graph.add_node(ego_username)

In [29]:
followers = load(followers_filename)

In [30]:
# Construct followers graph
for user in followers:
    follower_name = user.get('name')
    followers_count = user.get('public_metrics').get('followers_count')
    following_count = user.get('public_metrics').get('following_count')
    tweet_count = user.get('public_metrics').get('tweet_count')
    ego_graph.add_node(follower_name, followers_count=followers_count, following_count=following_count, tweet_count=tweet_count, color_id=calculate_importance(followers_count))
    ego_graph.add_edge(ego_username, follower_name)

In [31]:
# Get users that the ego follows
following = load(following_filename)

In [32]:
# Construct following graph
for user in following:
    following_name = user.get('name')
    followers_count = user.get('public_metrics').get('followers_count')
    following_count = user.get('public_metrics').get('following_count')
    tweet_count = user.get('public_metrics').get('tweet_count')
    ego_graph.add_node(following_name, followers_count=followers_count, following_count=following_count, tweet_count=tweet_count, color_id=calculate_importance(followers_count))
    ego_graph.add_edge(following_name, ego_username)

In [33]:
# Store the graph
with open(graph_filename, 'wb') as fOut:
    nx.write_graphml(ego_graph, fOut)